# Description

In [564]:
"""
List of cars, regulated pollutants
NEDC cold 23, WLTP cold 23 and RDE

""" 

'\nList of cars, regulated pollutants\nNEDC cold 23, WLTP cold 23 and RDE\n\n'

# Libraries

In [1]:
%matplotlib inline
import pandas as pd, datetime as dt
#from co2mpas.datasync import _get_interp_method
import collections
import numpy as np
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import glob, os
import time

## Definitions

In [14]:
car = {'Audi A1': ['./Audi A1/', 'xls',98,'gasoline'],             # we could get vela2 PN data from the analysis_veh.xls DB
      'BMW 530d': ['./BMW 530d/', 'xlsx',124,'diesel'],
      'Audi A3': ['./Audi A3/', 'xls',120,'diesel'],               # specify the FWD for NEDC in function lab_a1 
      'Citroen Cactus': ['./Citroen Cactus/','xls',95,'diesel'],   # specify WLTP_COLD in function lab_a1
      'Fiat 500x': ['./Fiat 500x/','xls',144,'diesel'],
      'Opel Astra': ['./Opel Astra/','',103,'gasoline'],           # Alternative code because of Vela8 file format (.xls for WLTP and .xlsx for NEDC)
      'Fiat Panda': ['./Fiat Panda/', 'xlsx',119,'gasoline'],
      'Kia Sportage': ['./Kia Sportage/','xlsx',124,'diesel'],
      'Peugeot Partner': ['./Peugeot Partner/','xls',112,'diesel'],
      'Renault Twingo': ['./Renault Twingo/', 'xlsx',112,'gasoline'],
      'VW Golf': ['./VW Golf/','xlsx',117,'diesel'],
      'Porsche Cayenne': ['./Porsche Cayenne/','xlsx',179,'diesel']
      #'Renault Captur':[] 
      #'VW Tiguan': ['./VW Tiguan/','xlsx',159,'diesel'],          # this remains OUT. not enough info and it'd require changes 
      }

In [523]:
vehicle = 'Citroen Cactus'

In [524]:
path = car[vehicle][0]
filetype = car[vehicle][1]
t_a = car[vehicle][2]
fuel = car[vehicle][3]

## Create the empty dataframe (ONLY BEFORE THE 1ST CAR!!!)

In [17]:
"""
###############################
Only to be used the first time!
###############################
"""

arrays = [['fuel','T_A',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE'],
           ['-', '-', 
           'CO2','CO2','CO2','CO2','CO2',             # reorder columns
           'NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN',
           'CO2','CO2','CO2','CO2','CO2','CO2','CO2',  
           'NOx','NOx','NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN','PN','PN',
           'CO2','CO2','CO2','CO2','CO2',  
           'NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN'],
           ['-', '-',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','urban','rural','motorway','300',
           'complete','urban','rural','motorway','300',
           'complete','urban','rural','motorway','300',
           'complete','urban','rural','motorway','300',
           'complete','urban','rural','motorway','300']
         ]

tuples = list(zip(*arrays))

my_columns = pd.MultiIndex.from_tuples(tuples, names=['cycle', 'pollutant','phase'])

my_index = []

RegPollEmi = pd.DataFrame(index=my_index, columns=my_columns)
#RegPollEmi.columns

# Vehicle

## Lab

### Distance based

#### Only for A3 

In [490]:
A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC_COLD*FWD*23*.' + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLTP_COLD*23*' + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

#### Only for Cactus

In [525]:
A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC_COLD*FWD*23*.' + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLTP_COLD*23*' + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

#### Most of cars

In [ ]:
A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC_COLD*23*.' + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLTP_COLD*23*' + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

#### Follow from here

In [526]:
def lab_A1(cycle, file, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = A1['cycle'][cycle]['phases'][phases][0]
        stop = A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=1, header=0)                          # read the proper sheets within each file
        print(time.time() - tic)
        
        v = df['DACtrl_ActualSpeed\n[kph]'][start:stop]                                  # speed (km/h)
        t_s = len(df.index[start:stop])                                                  # time(s)
        t_m = t_s/60                                                         # time (hours)
        t_h = t_s/3600                                                       # time (hours)
        v_mean = v.mean()                                                    # speed means
        d = v_mean * t_h                                                     # distance means
        
#         CO2_150 = df['c_CO2_Dil_g_vec\n[grams]'][0:150]
#         CO2_150d = CO2_150.sum()
        CO2 = df['c_CO2_Dil_g_vec\n[grams]'][start:stop]
        CO2_sum = CO2.sum()
        CO2km = CO2_sum/d                                                    # CO2 g/km 
        
        
        
        NOx = df['c_NOx_Dil_g_vec\n[grams]'][start:stop]
        NOx_sum = NOx.sum()*1000
        NOxkm = NOx_sum/d                                                    # NOx mg/km

        CO = df['c_CO_Dil_g_vec\n[grams]'][start:stop]
        CO_sum = CO.sum()*1000
        COkm = CO_sum/d                                                      # CO mg/km

        HC = df['c_THC_Dil_g_vec\n[grams]'][start:stop]
        HC_sum = HC.sum()*1000
        HCkm = HC_sum/d                                                      # HC mg/km

        flow_gas = df['c_Vmix_vec\n[m3/min]'][start:stop]                               # flow of gas mixture [m3/min]
        flow_avg = flow_gas.sum()/t_s                                       # average in m3/min
        vol_gas = flow_avg * t_m                                            # total volume of gas mixture (m3)
        vol_gas_cc = vol_gas * 1e6                                          # total volume of gas mixture (cm3)

        PN = df["Counter\n[cpcc]"][start:stop]                                         # column of PN flow in #/cm3
        try:
            PN_sum = PN.sum()                                                  
            PN_avg = PN_sum / t_s                                               # average of flow in #/cm3

            PN_total = PN_avg * vol_gas_cc                                      # total emitted in #
            PN_total_km = PN_total / d                                          # total emission in #/km
        except:
            PN_total_km = 0
        #key = each.split('T ')[1].split('.x')[0]                             # shortening the titles of the files

        #key = str(i) + "." + str(key)                                        # numbering the files
        dic_lab[each] = {
            'CO2km': CO2km,
            'NOxkm': NOxkm,
            'COkm': COkm,
            'HCkm': HCkm,
            'PNkm': PN_total_km,
            't_s': t_s, 
            'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

#### Alternative to handle vela8 files (NEDC Opel Astra)

In [255]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC*COLD_23*.xlsx',# + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '150/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLT*COLD*23*.xls',# + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '150/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

In [256]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


def lab_A2(cycle, file, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = A1['cycle'][cycle]['phases'][phases][0]
        stop = A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=0, header=None)

        print(time.time() - tic)
        
        
        vals = df.values
        mux = pd.MultiIndex.from_arrays(df.ffill(1).values[:2, 1:], names=[None, 'Time'])   # trick to create the proper MultiIndex
        df = pd.DataFrame(df.values[2:, 1:], df.values[2:, 0], mux)                         # same here

        df.drop(df.index[[0,1]],inplace=True)                                               # remove the 2 first rows after the headers

        CO2 = df['Diluted grams']['CO2'][start:stop]
        CO2_sum = CO2.sum()
        NOx = df['Diluted grams']['NOx'][start:stop]
        NOx_sum = NOx.sum()*1000                                                            # change unit to mg/km
        CO = df['Diluted grams']['CO-L'][start:stop]
        CO_sum = CO.sum()*1000
        HC = df['Diluted grams']['HC'][start:stop]
        HC_sum = HC.sum()*1000                                                            # change unit to mg/km
               
        PN = df['CVS']['PN']
        PN.columns = ['PN_cm3', 'PN_s']
        PN = PN.PN_s[start:stop]
        PN_sum = PN.sum()

        v = df['Chassis dyno']['Speed'][start:stop]                                   # group speed by intervals                    
        v_mean = v.mean()                                                  # speed means
        t_s = len(df.index[start:stop])                                                #  time(s)
        t_m = t_s/60
        t_h = t_s/3600                                                                    # time (hours)

        d = v_mean * t_h #b.reset_index(drop=True) * d.reset_index(drop=True)                # distance means  

        CO2km = CO2_sum/d                                                    # CO2 per km 
        NOxkm = NOx_sum/d                                                    # NOx / km                                                     
        COkm = CO_sum/d                                                    # NOx / km                                                  
        HCkm = HC_sum/d
        PNkm = PN_sum/d
        
        dic_lab[each] = {
                'CO2km': CO2km,
                'NOxkm': NOxkm,
                'COkm': COkm,
                'HCkm': HCkm,
                'PNkm': PNkm,
                't_s': t_s, 
                'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

In [257]:
dic_lab_all = {}

In [258]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    for phase in A1['cycle'][cycle]['phases']:
        if cycle == 'NEDC':
            #print(A1['cycle'][cycle])
            dic_lab_all[cycle][phase] = lab_A2(cycle, file, phase) 
            #[phase] = run_A1(cycle, file, phase)
        else:
            dic_lab_all[cycle][phase] = lab_A1(cycle, file, phase) 

17.92470908164978
16.855305910110474
17.601863622665405
16.222687482833862
19.299888610839844
16.599472284317017
17.40797185897827
17.089174270629883
0.5496859550476074
0.5357000827789307
0.5357027053833008
0.5509533882141113
0.5896787643432617
0.558692216873169
0.5616819858551025
0.5566942691802979
0.5716724395751953
0.5746612548828125
0.5796802043914795
0.6436302661895752
0.5476901531219482
0.544682502746582
0.550689697265625
0.5516953468322754
0.5316946506500244
0.5437073707580566
0.5377037525177002
0.5446887016296387
0.5426890850067139
0.6546401977539062
0.5436735153198242
0.5487024784088135
0.5536692142486572
0.5487222671508789
0.5466842651367188
0.5586795806884766
0.6346254348754883
0.5887134075164795


#### Follow from here

In [527]:
dic_lab_all = {}

In [528]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
        dic_lab_all[cycle][phase] = lab_A1(cycle, file, phase)

0.3867781162261963
0.3778500556945801
0.38977622985839844
0.632631778717041
0.3892180919647217
0.3767659664154053
0.3757915496826172
0.3727738857269287
0.3727853298187256
0.3807804584503174
0.3797755241394043
0.37578868865966797
0.3787655830383301
0.3767824172973633
0.37180590629577637
0.3767831325531006
0.3807828426361084
0.4937145709991455
0.37778496742248535
0.37178611755371094
0.5636749267578125
0.5576887130737305
0.5656700134277344
0.5606844425201416
0.5696704387664795
0.554699182510376
0.5616707801818848
0.5596828460693359
0.685579776763916
0.5466809272766113
0.5727102756500244
0.5507030487060547


In [529]:
a = pd.DataFrame.from_dict({(i,j,k): dic_lab_all[i][j][k] 
                           for i in dic_lab_all.keys() 
                           for j in dic_lab_all[i].keys()
                           for k in dic_lab_all[i][j].keys()},)
                       #orient='index')
a = a.replace(0, np.nan)
a

NEDC  \
                                                                     300/d   
      ././Citroen Cactus\VELA2_01072016_003_Modal_TP_NEDC_COLD_FWD_23C.xls   
CO2km                                       1.523360e+02                     
COkm                                        2.486576e+03                     
HCkm                                        1.315664e+02                     
NOxkm                                       7.777933e+01                     
PNkm                                        9.309611e+10                     
d                                           1.384545e+00                     
t_s                                         3.000000e+02                     

                                                                         \
                                                                          
      ././Citroen Cactus\VELA2_05072016_001_Modal_NEDC_COLD_FWD_23C.xls   
CO2km                                       1.503026e+02                  
COkm                                        2.781188e+03                  
HCkm                                        1.619611e+02                  
NOxkm                                       7.943288e+01                  
PNkm                                        8.897836e+09                  
d                                           1.380579e+00                  
t_s                                         3.000000e+02                  

                                                                         \
                                                                          
      ././Citroen Cactus\VELA2_06072016_004_Modal_NEDC_COLD_FWD_23C.xls   
CO2km                                       1.483280e+02                  
COkm                                        2.517211e+03                  
HCkm                                        1.402463e+02                  
NOxkm                                       6.882585e+01                  
PNkm                                        3.241584e+10                  
d                                           1.380855e+00                  
t_s                                         3.000000e+02                  

                                                                         \
                                                                          
      ././Citroen Cactus\VELA2_12072016_001_Modal_NEDC_COLD_FWD_23C.xls   
CO2km                                       1.422547e+02                  
COkm                                        2.449860e+03                  
HCkm                                        1.211284e+02                  
NOxkm                                       8.631145e+01                  
PNkm                                        6.738726e+09                  
d                                           1.376398e+00                  
t_s                                         3.000000e+02                  

                                                                            \
                                                                             
      ././Citroen Cactus\VELA2_30062016_001_Modal_TP_NEDC_COLD_FWD_23C.xls   
CO2km                                       1.557647e+02                     
COkm                                        2.412878e+03                     
HCkm                                        1.449267e+02                     
NOxkm                                       8.028754e+01                     
PNkm                                        2.132092e+11                     
d                                           1.368750e+00                     
t_s                                         3.000000e+02                     

                                                                            \
                                                                  complete   
      ././Citroen Cactus\VELA2_01072016_003_Modal_TP_NEDC_COLD_FWD_23C.xls   
CO2km                

In [530]:
# a.groupby(level=1).mean(axis=1)
a_avg =a.T.groupby(level=[0,1]).mean()       # mean of repeated tests
a_avg

CO2km         COkm        HCkm       NOxkm  \
NEDC 300/d       149.797212  2529.542783  139.965778   78.527412   
     complete    114.342562   342.415621   19.730133   29.155280   
     extraurban  104.667121     9.712553    0.240010   14.636739   
     urban       130.942092   913.245189   53.249824   54.059993   
WLTP 300/d       149.167682   341.094564    8.781724  232.139651   
     complete    143.448117    43.667772    0.784868   54.710928   
     extrahigh   167.068575    21.803184    0.002157   74.675755   
     high        125.328127     4.692947    0.000193    0.850190   
     low         147.409296   231.706689    5.791952  165.227119   
     medium      127.313109    18.239772    0.074825   29.644513   

                         PNkm          d     t_s  
NEDC 300/d       7.087155e+10   1.378225   300.0  
     complete    1.214421e+10  11.010835  1180.0  
     extraurban  9.679518e+08   6.956471   400.0  
     urban       3.133217e+10   4.054365   780.0  
WLTP 300/d       3.169623e+10   2.027210   300.0  
     complete    3.835728e+09  23.248383  1800.0  
     extrahigh   7.175318e+08   8.232475   323.0  
     high        9.188180e+08   7.180838   455.0  
     low         2.254897e+10   3.087899   589.0  
     medium      1.435464e+09   4.747170   433.0

In [531]:
a_avg = a_avg.drop(['d','t_s'], axis=1)      # remove the columns not to be used
a_avg = a_avg.unstack(level=[-1,0])          # put the 2 rows in 1 row (larger)
a_avg = pd.DataFrame(a_avg).T
a_avg = a_avg.swaplevel(0,2,axis=1)          # order Multiindex levels
a_avg = a_avg.swaplevel(1,2,axis=1)          # order Multiindex levels
a_avg = a_avg.sort_index(axis=1, level=0)    # sort by pollutant
# a_avg[a_avg.columns.set_levels([['complete', 'urban','extraurban','150/d','complete',
#                                  'low','medium','high','extrahigh','150/d']], level=2)]
LAB1 = a_avg

In [532]:
LAB1.index = [vehicle]                             # index name
LAB1.columns = LAB1.columns.droplevel(level=[0,1])    # no interested in Multiindex yet, so convert to regular df

LAB1.columns =['CO2_300/d_N','CO2_COMP_N','CO2_EU_N','CO2_URB_N',          # rename columns
               'CO_300/d_N','CO_COMP_N','CO_EU_N','CO_URB_N',
               'HC_300/d_N','HC_COMP_N','HC_EU_N','HC_URB_N',
               'NOx_300/d_N','NOx_COMP_N','NOx_EU_N','NOx_URB_N',
               'PN_300/d_N','PN_COMP_N','PN_EU_N','PN_URB_N',
               'CO2_300/d_W','CO2_COMP_W','CO2_EH_W','CO2_HIGH_W','CO2_LOW_W','CO2_MED_W',
               'CO_300/d_W','CO_COMP_W','CO_EH_W','CO_HIGH_W','CO_LOW_W','CO_MED_W',
               'HC_300/d_W','HC_COMP_W','HC_EH_W','HC_HIGH_W','HC_LOW_W','HC_MED_W',
               'NOx_300/d_W','NOx_COMP_W','NOx_EH_W','NOx_HIGH_W','NOx_LOW_W','NOx_MED_W',
               'PN_300/d_W','PN_COMP_W','PN_EH_W','PN_HIGH_W','PN_LOW_W','PN_MED_W']
              
LAB1 = LAB1[['CO2_COMP_N','CO2_URB_N','CO2_EU_N','CO2_300/d_N',             # reorder columns
            'NOx_COMP_N','NOx_URB_N','NOx_EU_N','NOx_300/d_N',
            'CO_COMP_N','CO_URB_N','CO_EU_N','CO_300/d_N',
            'HC_COMP_N','HC_URB_N','HC_EU_N','HC_300/d_N',
            'PN_COMP_N','PN_URB_N','PN_EU_N','PN_300/d_N',
            'CO2_COMP_W','CO2_LOW_W','CO2_MED_W','CO2_HIGH_W','CO2_EH_W','CO2_300/d_W',
            'NOx_COMP_W','NOx_LOW_W','NOx_MED_W','NOx_HIGH_W','NOx_EH_W','NOx_300/d_W',
            'CO_COMP_W','CO_LOW_W','CO_MED_W','CO_HIGH_W','CO_EH_W','CO_300/d_W',
            'HC_COMP_W','HC_LOW_W','HC_MED_W','HC_HIGH_W','HC_EH_W','HC_300/d_W',
            'PN_COMP_W','PN_LOW_W','PN_MED_W','PN_HIGH_W','PN_EH_W','PN_300/d_W']]

LAB1

,CO2_COMP_N,CO2_URB_N,CO2_EU_N,CO2_300/d_N,NOx_COMP_N,NOx_URB_N,NOx_EU_N,NOx_300/d_N,CO_COMP_N,CO_URB_N,...,HC_MED_W,HC_HIGH_W,HC_EH_W,HC_300/d_W,PN_COMP_W,PN_LOW_W,PN_MED_W,PN_HIGH_W,PN_EH_W,PN_300/d_W
Citroen Cactus,114.342562,130.942092,104.667121,149.797212,29.15528,54.059993,14.636739,78.527412,342.415621,913.245189,...,0.074825,0.000193,0.002157,8.781724,3.835728e+09,2.254897e+10,1.435464e+09,9.188180e+08,7.175318e+08,3.169623e+10


### 300 sec (non-distance based)

In [533]:
def lab_A1(cycle, file):#, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = 0 #A1['cycle'][cycle]['phases'][phases][0]
        stop = 300 #A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        df = pd.read_excel(each, sheetname=1, header=0)                          # read the proper sheets within each file
        print(time.time() - tic)
        
        v = df['DACtrl_ActualSpeed\n[kph]'][start:stop]                                  # speed (km/h)
        t_s = len(df.index[start:stop])                                                  # time(s)
        t_m = t_s/60                                                         # time (hours)
        t_h = t_s/3600                                                       # time (hours)
        v_mean = v.mean()                                                    # speed means
        d = v_mean * t_h                                                     # distance means
        
#         CO2_150 = df['c_CO2_Dil_g_vec\n[grams]'][0:150]
#         CO2_150d = CO2_150.sum()
        CO2 = df['c_CO2_Dil_g_vec\n[grams]'][start:stop]
        CO2_sum = CO2.sum()
        CO2km = CO2_sum/d                                                    # CO2 g/km 
        
        
        
        NOx = df['c_NOx_Dil_g_vec\n[grams]'][start:stop]
        NOx_sum = NOx.sum()*1000
        NOxkm = NOx_sum/d                                                    # NOx mg/km

        CO = df['c_CO_Dil_g_vec\n[grams]'][start:stop]
        CO_sum = CO.sum()*1000
        COkm = CO_sum/d                                                      # CO mg/km

        HC = df['c_THC_Dil_g_vec\n[grams]'][start:stop]
        HC_sum = HC.sum()*1000
        HCkm = HC_sum/d                                                      # HC mg/km

        flow_gas = df['c_Vmix_vec\n[m3/min]'][start:stop]                               # flow of gas mixture [m3/min]
        flow_avg = flow_gas.sum()/t_s                                       # average in m3/min
        vol_gas = flow_avg * t_m                                            # total volume of gas mixture (m3)
        vol_gas_cc = vol_gas * 1e6                                          # total volume of gas mixture (cm3)

        PN = df["Counter\n[cpcc]"][start:stop]                                         # column of PN flow in #/cm3
        try:
            PN_sum = PN.sum()                                                  
            PN_avg = PN_sum / t_s                                               # average of flow in #/cm3

            PN_total = PN_avg * vol_gas_cc                                      # total emitted in #
            PN_total_km = PN_total / d                                          # total emission in #/km
        except:
            PN_total = 0
            PN_total_km = 0

        #key = each.split('T ')[1].split('.x')[0]                             # shortening the titles of the files

        #key = str(i) + "." + str(key)                                        # numbering the files
        dic_lab[each] = {
            'CO2km': CO2_sum,
            'NOxkm': NOx_sum,
            'COkm': CO_sum,
            'HCkm': HC_sum,
            'PNkm': PN_total,
            't_s': t_s, 
            'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

#### Only for Astra

In [276]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""
def lab_A2(cycle, file):#, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = 0 #A1['cycle'][cycle]['phases'][phases][0]
        stop = 300 #A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=0, header=None)

        print(time.time() - tic)
        
        
        vals = df.values
        mux = pd.MultiIndex.from_arrays(df.ffill(1).values[:2, 1:], names=[None, 'Time'])   # trick to create the proper MultiIndex
        df = pd.DataFrame(df.values[2:, 1:], df.values[2:, 0], mux)                         # same here

        df.drop(df.index[[0,1]],inplace=True)                                               # remove the 2 first rows after the headers

        CO2 = df['Diluted grams']['CO2'][start:stop]
        CO2_sum = CO2.sum()
        NOx = df['Diluted grams']['NOx'][start:stop]
        NOx_sum = NOx.sum()*1000                                                            # change unit to mg/km
        CO = df['Diluted grams']['CO-L'][start:stop]
        CO_sum = CO.sum()*1000
        HC = df['Diluted grams']['HC'][start:stop]
        HC_sum = HC.sum()*1000                                                            # change unit to mg/km
               
        PN = df['CVS']['PN']
        PN.columns = ['PN_cm3', 'PN_s']
        PN = PN.PN_s[start:stop]
        PN_sum = PN.sum()

        v = df['Chassis dyno']['Speed'][start:stop]                                   # group speed by intervals                    
        v_mean = v.mean()                                                  # speed means
        t_s = len(df.index[start:stop])                                                #  time(s)
        t_m = t_s/60
        t_h = t_s/3600                                                                    # time (hours)

        d = v_mean * t_h #b.reset_index(drop=True) * d.reset_index(drop=True)                # distance means  

        CO2km = CO2_sum/d                                                    # CO2 per km 
        NOxkm = NOx_sum/d                                                    # NOx / km                                                     
        COkm = CO_sum/d                                                    # NOx / km                                                  
        HCkm = HC_sum/d
        PNkm = PN_sum/d

        dic_lab[each] = {
                'CO2km': CO2_sum,
                'NOxkm': NOx_sum,
                'COkm': CO_sum,
                'HCkm': HC_sum,
                'PNkm': PN_sum,
                't_s': t_s, 
                'd': d}                                         # save means in the dictionary
        i = i+1

    return dic_lab
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


'\nONLY FOR ASTRA!!\nONLY FOR ASTRA!!\nONLY FOR ASTRA!!\n'

In [277]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""

for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
    if cycle == 'NEDC':
            #print(A1['cycle'][cycle])
        dic_lab_all[cycle] = lab_A2(cycle, file) 
            #[phase] = run_A1(cycle, file, phase)
    else:
        dic_lab_all[cycle] = lab_A1(cycle, file)

17.415002584457397
16.98921251296997
0.5676920413970947
0.646611213684082
0.5726833343505859
0.5406899452209473
0.5826647281646729


#### Follow from here

In [534]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_lab_all[cycle]= lab_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

0.3947865962982178
0.3957700729370117
0.3788003921508789
0.3737828731536865
0.37279772758483887
0.5576798915863037
0.5516839027404785


In [535]:
b = pd.DataFrame.from_dict({(i,j): dic_lab_all[i][j] 
                           for i in dic_lab_all.keys() 
                           for j in dic_lab_all[i].keys()},)
                       #orient='index')
b = b.replace(0,np.nan)
b

NEDC  \
      ././Citroen Cactus\VELA2_01072016_003_Modal_TP_NEDC_COLD_FWD_23C.xls   
CO2km                                       2.109162e+02                     
COkm                                        3.442777e+03                     
HCkm                                        1.821596e+02                     
NOxkm                                       1.076890e+02                     
PNkm                                        1.288958e+11                     
d                                           1.384545e+00                     
t_s                                         3.000000e+02                     

                                                                         \
      ././Citroen Cactus\VELA2_05072016_001_Modal_NEDC_COLD_FWD_23C.xls   
CO2km                                       2.075046e+02                  
COkm                                        3.839650e+03                  
HCkm                                        2.236000e+02                  
NOxkm                                       1.096633e+02                  
PNkm                                        1.228416e+10                  
d                                           1.380579e+00                  
t_s                                         3.000000e+02                  

                                                                         \
      ././Citroen Cactus\VELA2_06072016_004_Modal_NEDC_COLD_FWD_23C.xls   
CO2km                                       2.048195e+02                  
COkm                                        3.475903e+03                  
HCkm                                        1.936598e+02                  
NOxkm                                       9.503850e+01                  
PNkm                                        4.476157e+10                  
d                                           1.380855e+00                  
t_s                                         3.000000e+02                  

                                                                         \
      ././Citroen Cactus\VELA2_12072016_001_Modal_NEDC_COLD_FWD_23C.xls   
CO2km                                       1.957991e+02                  
COkm                                        3.371983e+03                  
HCkm                                        1.667209e+02                  
NOxkm                                       1.187989e+02                  
PNkm                                        9.275170e+09                  
d                                           1.376398e+00                  
t_s                                         3.000000e+02                  

                                                                            \
      ././Citroen Cactus\VELA2_30062016_001_Modal_TP_NEDC_COLD_FWD_23C.xls   
CO2km                                       2.132029e+02                     
COkm                                        3.302627e+03                     
HCkm                                        1.983684e+02                     
NOxkm                                       1.098936e+02                     
PNkm                                        2.918301e+11                     
d                                           1.368750e+00                     
t_s                                         3.000000e+02                     

                                                                  WLTP  \
      ././Citroen Cactus\VELA2_05072016_004_Modal_WLTP_COLD_FW_23C.xls   
CO2km                                       2.960667e+02                 
COkm                                        6.677720e+02                 
HCkm                                        1.829064e+01                 
NOxkm                                       4.306426e+02                 
PNkm                                        8.987477e+09                 
d                                           2.013398e+00                 
t_s                                         

In [536]:
b_avg = b.T.groupby(level=0).mean()
b_avg = b_avg.drop(['d','t_s'], axis=1)
b_avg

,CO2km,COkm,HCkm,NOxkm,PNkm
NEDC,206.448435,3486.587810,192.901747,108.216668,9.740935e+10
WLTP,302.423427,691.600418,17.798214,470.847802,6.463102e+10


In [537]:
NEDC300 = pd.DataFrame(b_avg.loc['NEDC',:])
NEDC300 = NEDC300.T
NEDC300.index = [vehicle]
WLTP300 = pd.DataFrame(b_avg.loc['WLTP',:])
WLTP300 = WLTP300.T
WLTP300.index = [vehicle]
WLTP300

LAB2 = pd.concat([NEDC300,WLTP300],axis=1)
LAB2.columns = ['CO2300_N','CO300_N','HC300_N','NOx300_N','PN300_N',
               'CO2300_W','CO300_W','HC300_W','NOx300_W','PN300_W']
LAB2 = LAB2[['CO2300_N','NOx300_N','CO300_N','HC300_N','PN300_N',
               'CO2300_W','NOx300_W','CO300_W','HC300_W','PN300_W']]
LAB2

,CO2300_N,NOx300_N,CO300_N,HC300_N,PN300_N,CO2300_W,NOx300_W,CO300_W,HC300_W,PN300_W
Citroen Cactus,206.448435,108.216668,3486.58781,192.901747,9.740935e+10,302.423427,470.847802,691.600418,17.798214,6.463102e+10


In [538]:
LAB = pd.concat([LAB1,LAB2],axis=1)
LAB = LAB[[
           'CO2_COMP_N','CO2_URB_N','CO2_EU_N','CO2_300/d_N','CO2300_N',             # reorder columns
           'NOx_COMP_N','NOx_URB_N','NOx_EU_N','NOx_300/d_N','NOx300_N',
           'CO_COMP_N','CO_URB_N','CO_EU_N','CO_300/d_N','CO300_N',
           'HC_COMP_N','HC_URB_N','HC_EU_N','HC_300/d_N','HC300_N',
           'PN_COMP_N','PN_URB_N','PN_EU_N','PN_300/d_N','PN300_N',
           'CO2_COMP_W','CO2_LOW_W','CO2_MED_W','CO2_HIGH_W','CO2_EH_W','CO2_300/d_W','CO2300_W',
           'NOx_COMP_W','NOx_LOW_W','NOx_MED_W','NOx_HIGH_W','NOx_EH_W','NOx_300/d_W','NOx300_W',
           'CO_COMP_W','CO_LOW_W','CO_MED_W','CO_HIGH_W','CO_EH_W','CO_300/d_W','CO300_W',
           'HC_COMP_W','HC_LOW_W','HC_MED_W','HC_HIGH_W','HC_EH_W','HC_300/d_W','HC300_W',
           'PN_COMP_W','PN_LOW_W','PN_MED_W','PN_HIGH_W','PN_EH_W','PN_300/d_W','PN300_W'
    ]]
LAB

,CO2_COMP_N,CO2_URB_N,CO2_EU_N,CO2_300/d_N,CO2300_N,NOx_COMP_N,NOx_URB_N,NOx_EU_N,NOx_300/d_N,NOx300_N,...,HC_EH_W,HC_300/d_W,HC300_W,PN_COMP_W,PN_LOW_W,PN_MED_W,PN_HIGH_W,PN_EH_W,PN_300/d_W,PN300_W
Citroen Cactus,114.342562,130.942092,104.667121,149.797212,206.448435,29.15528,54.059993,14.636739,78.527412,108.216668,...,0.002157,8.781724,17.798214,3.835728e+09,2.254897e+10,1.435464e+09,9.188180e+08,7.175318e+08,3.169623e+10,6.463102e+10


## RDE

In [539]:
A1 = {
    'cycle': {
        'Esperia': {'file': './'+path+'/*ESP*.xlsx'},
        'Labiena': {'file': './'+path+'/*LAB*.xlsx'}, 
             },
    
     }

### Distance based

In [540]:
def road_A1(cycle, file):
    files_RDE = glob.glob(file)                                         # read all EMROAD files
    dic_RDE = {}                                                            # empty dictionary road test

    i = 1
    for each in files_RDE:

        tic = time.time()
        df = pd.read_excel(each, sheetname='TEST REPORT', header=6)             # read the proper sheets within each file
        print(time.time() - tic)
        
        df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)   # convert the headers to read them: ' ' for '_'
        df.rename(columns=lambda x: x.strip().replace(".", "_"), inplace=True)   # convert the headers to read them: '.' for '_'


        CO2_COMP = df.COMPLETE_TEST['CO2'][1]      # g/km
        CO2_URB = df.URBAN['CO2'][1]               # g/km
        CO2_RUR = df.RURAL['CO2'][1]               # g/km
        CO2_MOT = df.MOTORW_['CO2'][1]             # g/km
        #CO2 = [CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT]

        NOx_COMP = df.COMPLETE_TEST['NOx'][1]*1000
        NOx_URB = df.URBAN['NOx'][1]*1000
        NOx_RUR = df.RURAL['NOx'][1]*1000
        NOx_MOT = df.MOTORW_['NOx'][1]*1000

        CO_COMP = df.COMPLETE_TEST['CO'][1]*1000      # g/km
        CO_URB = df.URBAN['CO'][1]*1000               # g/km
        CO_RUR = df.RURAL['CO'][1]*1000               # g/km
        CO_MOT = df.MOTORW_['CO'][1]*1000             # g/km

        HC_COMP = df.COMPLETE_TEST['THC'][1]*1000      # g/km
        HC_URB = df.URBAN['THC'][1]*1000               # g/km
        HC_RUR = df.RURAL['THC'][1]*1000               # g/km
        HC_MOT = df.MOTORW_['THC'][1]*1000             # g/km

        PN_COMP = df.COMPLETE_TEST['PN'][1]      # g/km
        PN_URB = df.URBAN['PN'][1]               # g/km
        PN_RUR = df.RURAL['PN'][1]               # g/km
        PN_MOT = df.MOTORW_['PN'][1]             # g/km

        #key = each.split('T ')[1].split('.x')[0]
        ###########################################       each = str(i) + "_" + str(each)
        dic_RDE[each] = {'CO2_COMP':CO2_COMP,'CO2_URB':CO2_URB,'CO2_RUR':CO2_RUR,'CO2_MOT':CO2_MOT,
                          'NOx_COMP':NOx_COMP,'NOx_URB':NOx_URB,'NOx_RUR':NOx_RUR,'NOx_MOT':NOx_MOT,
                          'CO_COMP':CO_COMP,'CO_URB':CO_URB,'CO_RUR':CO_RUR,'CO_MOT':CO_MOT,
                          'HC_COMP':HC_COMP,'HC_URB':HC_URB,'HC_RUR':HC_RUR,'HC_MOT':HC_MOT,
                          'PN_COMP':PN_COMP,'PN_URB':PN_URB,'PN_RUR':PN_RUR,'PN_MOT':PN_MOT,
                        }                                               # save means in the dictionary
        i = i+1
    return dic_RDE

In [541]:
dic_road_all = {}

In [542]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_road_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_road_all[cycle]= road_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

13.745081186294556
16.13997530937195
15.11614203453064
14.517651081085205
16.27964973449707
16.276635885238647
16.48651885986328
16.01880407333374


In [543]:
c = pd.DataFrame.from_dict({(i,j): dic_road_all[i][j] 
                           for i in dic_road_all.keys() 
                           for j in dic_road_all[i].keys()},)
                       #orient='index')

c = c.replace(0,np.nan)                  # We put NaN instead of 0 so that those are not considered in the .mean()
c.T

CO2_COMP  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  117.985222   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  119.551407   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  120.900589   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  127.296716   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  135.675119   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  135.587369   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  130.071683   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  154.382539   

                                                               CO2_MOT  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  135.051214   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  137.455698   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  132.463132   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  140.768259   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  143.987892   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  146.399864   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  148.504152   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  154.386947   

                                                               CO2_RUR  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  102.276445   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  102.052041   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  114.234442   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  121.686802   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  115.257651   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  119.834242   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  106.513911   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  152.352989   

                                                               CO2_URB  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  114.999399   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  117.193040   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  115.397952   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  118.155502   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  141.472448   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  133.318106   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  123.692578   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  155.424813   

                                                               CO_COMP  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...   31.436863   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...   46.693675   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...   71.097595   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...   16.524425   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...   64.180176   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...   16.350751   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...   83.273701   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  343.887267   

                                                                CO_MOT  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...         NaN   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...   19.918229   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...   41.951971   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...    2.853006   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...   31.195548   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...    0.338315   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...   61.814641   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  100.929100   

                                                            

In [544]:
c_avg = c.T.groupby(level=0).mean()
c_avg = c_avg[['CO2_COMP','CO2_URB','CO2_RUR','CO2_MOT',
              'NOx_COMP','NOx_URB','NOx_RUR','NOx_MOT',
              'CO_COMP','CO_URB','CO_RUR','CO_MOT',
              'HC_COMP','HC_URB','HC_RUR','HC_MOT',
              'PN_COMP','PN_URB','PN_RUR','PN_MOT']]
c_avg

,CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT,NOx_COMP,NOx_URB,NOx_RUR,NOx_MOT,CO_COMP,CO_URB,CO_RUR,CO_MOT,HC_COMP,HC_URB,HC_RUR,HC_MOT,PN_COMP,PN_URB,PN_RUR,PN_MOT
Esperia,121.433483,116.436473,110.062433,136.434575,419.648304,181.576488,526.155259,604.512179,41.438140,80.755037,23.280258,21.574402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Labiena,138.929177,138.476986,123.489698,148.319714,503.138979,361.526775,630.079824,557.238579,126.922974,208.927415,167.675826,48.569401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [545]:
RDE1 = pd.DataFrame(c_avg.mean())
RDE1.columns = [vehicle]
RDE1 = RDE1.T
RDE1

,CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT,NOx_COMP,NOx_URB,NOx_RUR,NOx_MOT,CO_COMP,CO_URB,CO_RUR,CO_MOT,HC_COMP,HC_URB,HC_RUR,HC_MOT,PN_COMP,PN_URB,PN_RUR,PN_MOT
Citroen Cactus,130.18133,127.45673,116.776065,142.377145,461.393642,271.551632,578.117541,580.875379,84.180557,144.841226,95.478042,35.071902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### RDE 300 sec (in g)

In [546]:
def road_A1(cycle, file):
    files_RDE = glob.glob(file)                                         # read all EMROAD files
    dic_RDE = {}                                                            # empty dictionary road test

    i = 1
    for each in files_RDE:

        tic = time.time()
        df = pd.read_excel(each, sheetname='CALCULATED DATA', header=1,skiprows=[2])
        print(time.time() - tic)
        
        df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)   # convert the headers to read them: ' ' for '_'
        df.rename(columns=lambda x: x.strip().replace(".", "_"), inplace=True)   # convert the headers to read them: '.' for '_'
    
        min_ind = df.loc[(df["Inst__Mass_CO2"] > 0.1)].index[0] # minimum index, engine turned on
           #  & (df["Inst__Mass_NOx"] > 0.000001)           # threshold chosen by looking the files
    
        CO2_300 = df.Inst__Mass_CO2[min_ind:min_ind+300].sum()           # grams of CO2
        NOx_300 = df.Inst__Mass_NOx[min_ind:min_ind+300].sum()*1000      # miligrams of NOx
        CO_300 = df.Inst__Mass_CO[min_ind:min_ind+300].sum()*1000       # miligrams of CO
        HC_300 = df.Inst__Mass_THC[min_ind:min_ind+300].sum()*1000      # miligrams of HC
        PN_300 = df.Inst__PN[min_ind:min_ind+300].sum()                 # total PN
        
        dic_RDE[each] = {
            'CO2 300': CO2_300,
            'NOx 300': NOx_300,
            'CO 300': CO_300,
            'HC 300': HC_300,
            'PN 300': PN_300
        }                                              # save means in the dictionary
        
        i = i+1
        
    return dic_RDE

In [547]:
dic_road_all = {}

In [548]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_road_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_road_all[cycle]= road_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

15.226254940032959
15.251230001449585
15.051345348358154
14.588626146316528
16.339602947235107
16.634414672851562
16.187685251235962
26.624704837799072


In [549]:
d = pd.DataFrame.from_dict({(i,j): dic_road_all[i][j] 
                           for i in dic_road_all.keys() 
                           for j in dic_road_all[i].keys()},)
                       #orient='index')
d = d.replace(0,np.nan)
d

Esperia  \
        ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_Test1.xlsx   
CO 300                                         2394.029267          
CO2 300                                         177.180875          
HC 300                                                 NaN          
NOx 300                                         449.348674          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_Test1.xlsx   
CO 300                                         2041.136560          
CO2 300                                         261.217826          
HC 300                                                 NaN          
NOx 300                                         453.699495          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_Test1.xlsx   
CO 300                                         1349.738724          
CO2 300                                         292.811218          
HC 300                                                 NaN          
NOx 300                                         527.002472          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_Test2.xlsx   
CO 300                                          535.569800          
CO2 300                                         225.616876          
HC 300                                                 NaN          
NOx 300                                         664.026639          
PN 300                                                 NaN          

                                                          Labiena  \
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_Test1.xlsx   
CO 300                                         1502.704877          
CO2 300                                         280.564514          
HC 300                                                 NaN          
NOx 300                                         558.979506          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_Test2.xlsx   
CO 300                                          427.992143          
CO2 300                                         293.004226          
HC 300                                                 NaN          
NOx 300                                         808.017974          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_Test1.xlsx   
CO 300                                          900.411417          
CO2 300                                         271.632339          
HC 300                                                 NaN          
NOx 300                                         676.741891          
PN 300                                                 NaN          

                                                                   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_Test2.xlsx  
CO 300                                          254.829324         
CO2 300                                         325.023221         
HC 300                                                 NaN         
NOx 300                                        1037.343284         
PN 300                                                 NaN

In [550]:
d_avg = d.T.groupby(level=0).mean()
d_avg = d_avg[['CO2 300','NOx 300','CO 300','HC 300','PN 300']]
d_avg

,CO2 300,NOx 300,CO 300,HC 300,PN 300
Esperia,239.206699,523.519320,1580.118588,NaN,NaN
Labiena,292.556075,770.270664,771.484440,NaN,NaN


In [551]:
e = pd.DataFrame(d_avg.mean())
e.columns = [vehicle]
RDE2 = e.T
RDE2

,CO2 300,NOx 300,CO 300,HC 300,PN 300
Citroen Cactus,265.881387,646.894992,1175.801514,NaN,NaN


In [552]:
RDE = pd.concat([RDE1,RDE2],axis=1)
RDE = RDE[['CO2_COMP','CO2_URB','CO2_RUR','CO2_MOT','CO2 300',
             'NOx_COMP','NOx_URB','NOx_RUR','NOx_MOT','NOx 300',
             'CO_COMP','CO_URB','CO_RUR','CO_MOT','CO 300',
             'HC_COMP','HC_URB','HC_RUR','HC_MOT','HC 300',
             'PN_COMP','PN_URB','PN_RUR','PN_MOT','PN 300']]
RDE

,CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT,CO2 300,NOx_COMP,NOx_URB,NOx_RUR,NOx_MOT,NOx 300,...,HC_COMP,HC_URB,HC_RUR,HC_MOT,HC 300,PN_COMP,PN_URB,PN_RUR,PN_MOT,PN 300
Citroen Cactus,130.18133,127.45673,116.776065,142.377145,265.881387,461.393642,271.551632,578.117541,580.875379,646.894992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Concat lab and road

In [553]:
veh = pd.concat([LAB,RDE], axis=1)#, ignore_index=True)
veh.insert(0, "T_A", t_a)
veh.insert(0,"fuel",fuel)
# AUDI_A1.columns =db_regpoll.columns
#veh.drop('Peugeot Partner', inplace=True)
veh

,fuel,T_A,CO2_COMP_N,CO2_URB_N,CO2_EU_N,CO2_300/d_N,CO2300_N,NOx_COMP_N,NOx_URB_N,NOx_EU_N,...,HC_COMP,HC_URB,HC_RUR,HC_MOT,HC 300,PN_COMP,PN_URB,PN_RUR,PN_MOT,PN 300
Citroen Cactus,diesel,95,114.342562,130.942092,104.667121,149.797212,206.448435,29.15528,54.059993,14.636739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [554]:
veh.columns = my_columns
RegPollEmi = RegPollEmi.append(veh)

In [555]:
RegPollEmi                             #.sort_index()#.drop('Peugeot Partner', inplace=True)

cycle                fuel  T_A        NEDC                          \
pollutant               -    -         CO2                           
phase                   -    -    complete       urban  extraurban   
Audi A1          gasoline   98  115.010052  141.499340   99.544178   
BMW 530d           diesel  124  142.593498  183.514712  118.979382   
Audi A3            diesel  120  156.967983  182.514809  142.272442   
Citroen Cactus     diesel   95  115.231531  129.569982  106.759973   
Fiat 500x          diesel  144  191.497721  215.876307  177.539683   
Opel Astra       gasoline  103  130.524707  162.965505  111.592557   
Fiat Panda       gasoline  119  129.183095  169.412697  105.658152   
Kia Sportage       diesel  124  130.444277  157.244714  114.835875   
Peugeot Partner    diesel  112  116.523647  130.515602  108.385462   
Renault Twingo   gasoline  112  120.245945  155.692207   99.469361   
VW Golf            diesel  117  122.991381  152.002223  105.176443   
Audi A3            diesel  120  151.025535  177.388071  135.889201   
Citroen Cactus     diesel   95  114.342562  130.942092  104.667121   

cycle                                                                       \
pollutant                                       NOx                          
phase                 300/d         300    complete      urban  extraurban   
Audi A1          170.124042  235.608907   11.489641  24.933212    3.640121   
BMW 530d         204.821367  280.360502   15.070664  20.150810   12.136080   
Audi A3          208.641299  283.021930   84.423641  75.503266   89.571212   
Citroen Cactus   144.263730  216.931869   26.808684  50.821858   12.359546   
Fiat 500x        243.125268  328.664990  196.017402  78.637758  263.499495   
Opel Astra       195.682618  271.416752   10.096388  25.069613    1.360553   
Fiat Panda       199.757413  276.063497    4.073624   7.087758    2.310302   
Kia Sportage     185.116731  254.598819   38.779593  54.625389   29.551978   
Peugeot Partner  145.856678  200.044691   13.569765  34.975806    1.119893   
Renault Twingo   180.784348  251.310183    4.597819   9.416676    1.773294   
VW Golf          178.102152  238.340758   17.106838  21.802389   14.144599   
Audi A3          199.679152  269.557351   67.007396  66.526807   67.285514   
Citroen Cactus   149.797212  206.448435   29.155280  54.059993   14.636739   

cycle                ...           RDE                                         \
pollutant            ...            HC                                     PN   
phase                ...      complete urban rural motorway 300      complete   
Audi A1              ...           NaN   NaN   NaN      NaN NaN  1.148899e+12   
BMW 530d             ...           NaN   NaN   NaN      NaN NaN  7.359687e+10   
Audi A3              ...           NaN   NaN   NaN      NaN NaN           NaN   
Citroen Cactus       ...           NaN   NaN   NaN      NaN NaN           NaN   
Fiat 500x            ...           NaN   NaN   NaN      NaN NaN           NaN   
Opel Astra           ...           NaN   NaN   NaN      NaN NaN  2.361732e+12   
Fiat Panda           ...           NaN   NaN   NaN      NaN NaN  4.645039e+11   
Kia Sportage         ...           NaN   NaN   NaN      NaN NaN  4.955236e+09   
Peugeot Partner      ...           NaN   NaN   NaN      NaN NaN           NaN   
Renault Twingo       ...           NaN   NaN   NaN      NaN NaN  6.668188e+10   
VW Golf              ...           NaN   NaN   NaN      NaN NaN           NaN   
Audi A3              ...           NaN   NaN   NaN      NaN NaN           NaN   
Citroen Cactus       ...           NaN   NaN   NaN      NaN NaN           NaN   

cycle                                                                    
pollutant                                                                
phase                   urban         rural      motorway           300  
Audi A1          1.953222e+12  5.146943e+11  7.160030e+11  2.234121e+13  
BMW 530d         1.190758e+11  8.9782

# Excel and DataFrame

In [556]:
RegPollEmi.to_excel('veh_3.xlsx')

In [558]:
dfr = pd.read_excel('veh_3.xlsx',header=(0,1,2))
dfr

cycle                fuel  T_A        NEDC                          \
pollutant               -    -         CO2                           
phase                   -    -    complete       urban  extraurban   
Audi A1          gasoline   98  115.010052  141.499340   99.544178   
BMW 530d           diesel  124  142.593498  183.514712  118.979382   
Audi A3            diesel  120  151.025535  177.388071  135.889201   
Citroen Cactus     diesel   95  114.342562  130.942092  104.667121   
Fiat 500x          diesel  144  191.497721  215.876307  177.539683   
Opel Astra       gasoline  103  130.524707  162.965505  111.592557   
Fiat Panda       gasoline  119  129.183095  169.412697  105.658152   
Kia Sportage       diesel  124  130.444277  157.244714  114.835875   
Peugeot Partner    diesel  112  116.523647  130.515602  108.385462   
Renault Twingo   gasoline  112  120.245945  155.692207   99.469361   
VW Golf            diesel  117  122.991381  152.002223  105.176443   

cycle                                                                       \
pollutant                                       NOx                          
phase                 300/d         300    complete      urban  extraurban   
Audi A1          170.124042  235.608907   11.489641  24.933212    3.640121   
BMW 530d         204.821367  280.360502   15.070664  20.150810   12.136080   
Audi A3          199.679152  269.557351   67.007396  66.526807   67.285514   
Citroen Cactus   149.797212  206.448435   29.155280  54.059993   14.636739   
Fiat 500x        243.125268  328.664990  196.017402  78.637758  263.499495   
Opel Astra       195.682618  271.416752   10.096388  25.069613    1.360553   
Fiat Panda       199.757413  276.063497    4.073624   7.087758    2.310302   
Kia Sportage     185.116731  254.598819   38.779593  54.625389   29.551978   
Peugeot Partner  145.856678  200.044691   13.569765  34.975806    1.119893   
Renault Twingo   180.784348  251.310183    4.597819   9.416676    1.773294   
VW Golf          178.102152  238.340758   17.106838  21.802389   14.144599   

cycle                ...           RDE                                         \
pollutant            ...            HC                                     PN   
phase                ...      complete urban rural motorway 300      complete   
Audi A1              ...           NaN   NaN   NaN      NaN NaN  1.148899e+12   
BMW 530d             ...           NaN   NaN   NaN      NaN NaN  7.359687e+10   
Audi A3              ...           NaN   NaN   NaN      NaN NaN           NaN   
Citroen Cactus       ...           NaN   NaN   NaN      NaN NaN           NaN   
Fiat 500x            ...           NaN   NaN   NaN      NaN NaN           NaN   
Opel Astra           ...           NaN   NaN   NaN      NaN NaN  2.361732e+12   
Fiat Panda           ...           NaN   NaN   NaN      NaN NaN  4.645039e+11   
Kia Sportage         ...           NaN   NaN   NaN      NaN NaN  4.955236e+09   
Peugeot Partner      ...           NaN   NaN   NaN      NaN NaN           NaN   
Renault Twingo       ...           NaN   NaN   NaN      NaN NaN  6.668188e+10   
VW Golf              ...           NaN   NaN   NaN      NaN NaN           NaN   

cycle                                                                    
pollutant                                                                
phase                   urban         rural      motorway           300  
Audi A1          1.953222e+12  5.146943e+11  7.160030e+11  2.234121e+13  
BMW 530d         1.190758e+11  8.978261e+10  5.733847e+09  3.960718e+11  
Audi A3                   NaN           NaN           NaN           NaN  
Citroen Cactus            NaN           NaN           NaN           NaN  
Fiat 500x                 NaN           NaN           NaN           NaN  
Opel Astra       2.764658e+12  1.539987e+12  2.624206e+12  1.200500e+13  
Fiat Panda       4.010631e+11  3.927972e+11  6.088371e+11  1.408203e+12  
Kia Sportage     1.184077e+10  2.273840e+08  8.4407

## Extra stuff (testing)

In [559]:
RegPollEmi = dfr     # if values changed manually in the veh Excel, then run the upper block
                     # and then this one, to copy it into the dataframe

In [561]:
RegPollEmi.NEDC.CO2

phase,complete,urban,extraurban,300/d,300
Audi A1,115.010052,141.499340,99.544178,170.124042,235.608907
BMW 530d,142.593498,183.514712,118.979382,204.821367,280.360502
Audi A3,151.025535,177.388071,135.889201,199.679152,269.557351
Citroen Cactus,114.342562,130.942092,104.667121,149.797212,206.448435
Fiat 500x,191.497721,215.876307,177.539683,243.125268,328.664990
Opel Astra,130.524707,162.965505,111.592557,195.682618,271.416752
Fiat Panda,129.183095,169.412697,105.658152,199.757413,276.063497
Kia Sportage,130.444277,157.244714,114.835875,185.116731,254.598819
Peugeot Partner,116.523647,130.515602,108.385462,145.856678,200.044691
Renault Twingo,120.245945,155.692207,99.469361,180.784348,251.310183
